In [1]:
#!voila test_chatbot.ipynb

In [2]:
#!pip install azure-ai-textanalytics

In [3]:
#!pip install pandas

In [4]:
#!pip install sklearn

In [5]:
#!pip install spacy-nightly --pre --user

In [6]:
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download de_core_news_lg

In [7]:
#!python -m spacy download de_core_news_lg

In [8]:
#!pip install spacy

In [9]:
#!pip install gensim

In [1]:
#wigets and display widgets
import ipywidgets as wg
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

#set working directory right
import os
if not str(os.getcwd()).endswith('Modules'):
    #print("Current working directory: {0}".format(os.getcwd()))
    os.chdir(os.getcwd() + '/Modules')
    #print("New working directory: {0}".format(os.getcwd()))
    
#Chatbot
from keyword_check import Keyword_check
from chatbot import Chatbot
from solrhandler import SolrHandler
from clusterer import Clusterer
from topicdeterminator import TopicDeterminator

Current working directory: C:\Users\aechs\Documents\Silvio\PJ-DS-Chatbot\CodingDirectory\Clustering
New working directory: C:\Users\aechs\Documents\Silvio\PJ-DS-Chatbot\CodingDirectory\Clustering\Modules


C:\Users\aechs\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:693: UserWarning: [W095] Model 'de_core_news_lg' (3.0.0) requires spaCy >=3.0.0,<3.1.0 and is incompatible with the current version (3.0.0rc5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
chatbots = [
    {"app":Chatbot,
    "kwargs":{}},
    {"app":Keyword_check,
    "kwargs":{}},
    {"app":Keyword_check,
    "kwargs":{"cluster_keywords":False}}]
maxResultSetSize = 3
activ_chatbot = None

In [45]:
radio = wg.RadioButtons(
    options=[("{0} {1}".format(chatbots[i]["app"].__name__, " ".join(chatbots[i]["kwargs"].keys())), i) for i in range(len(chatbots))],
    layout={'width': 'max-content'}, # If the items' names are long
    description='Chatbot Selection:',
    style={'description_width': 'initial'},
    disabled=False
)

In [46]:
display(radio)

RadioButtons(description='Chatbot Selection:', layout=Layout(width='max-content'), options=(('Chatbot ', 0), (…

In [5]:
def initChatbot(msg):
    global maxResultSetSize
    global activ_chatbot
    global next_functioncall
    print("Loading {0} ...".format(chatbots[radio.value]["app"].__name__))
    activ_chatbot = chatbots[radio.value]["app"](SolrHandler, Clusterer, TopicDeterminator, msg, maxResultSetSize, **chatbots[radio.value]["kwargs"])
    next_functioncall = answerQuestion
    return activ_chatbot.generateQuestion()

In [6]:
next_functioncall = initChatbot

In [7]:
def answerQuestion(msg):
    global activ_chatbot
    global maxResultSize
    if not msg:
        return "Bitte stellen Sie Ihre Frage."
    if msg.lower() in ["ja", "nein"]:
        activ_chatbot.refineResultset(msg.lower()=="ja")
        if activ_chatbot.isFinished():
            global next_functioncall
            next_functioncall = initChatbot
            return activ_chatbot.get_result_string()
        return activ_chatbot.generateQuestion()
    else:
        activ_chatbot.add_query(msg)
        return activ_chatbot.generateQuestion()

In [8]:
wg.HTML(
    value="Hallo mein Name ist <b>Nobbi</b> (noch besserer Bobbi), wie kann ich Ihnen behilflich sein? ",
    placeholder='',
    description='',
)

HTML(value='Hallo mein Name ist <b>Nobbi</b> (noch besserer Bobbi), wie kann ich Ihnen behilflich sein? ', pla…

In [10]:
# ui = wg.HBox([a, b, c])
# def f(a, b, c):
#     print((a, b, c))

# out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

# display(ui, out)

In [11]:
#interact(lambda Frage: next_functioncall(Frage), Frage="Wohnung", continuous_update=False);

In [44]:
bot = interactive(lambda Frage: next_functioncall(Frage),
                  {'manual': True, "manual_name":"senden", "manual_description":'Frage:'},
                  Frage=wg.Combobox(options=["ja", "nein"],
                                    default=""), continuous_update=False);
#bot.widget.children[0].description = 'Frage:'
display(bot)

AttributeError: 'interactive' object has no attribute 'widget'

In [ ]:
#print(activ_chatbot.df["d115Description"].to_string())

In [ ]:
#widgetLink = widgets.jslink((floatTextBox, 'value'), (floatSlider, 'value'))